02_feature_engineering_pipeline

Objective:
    1. Reading the file from the loacation
    2. Doing data preprcessing like extracting relevant features, maping values.
    3. Let's do 1 and 2

feature_engineering: \
  root_dir: artifacts/feature_engineering  \
  training_data_path: artifacts/feature_engineering  \
  training_data_file: artifacts/feature_engineering/tweets.cs


In [63]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    training_data_path: Path
    training_data_file: str
    training_cleansed_data: str
    

In [2]:
%pwd

'c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT\\research'

In [3]:
import os
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT'

In [64]:
from pathlib import Path

CONFIG_FILE_PATH = Path("c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT/config/config.yaml")
PARAMS_FILE_PATH = Path("c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT/params.yaml")

In [65]:
from src.textClassifier.constants import *
from src.textClassifier.utils.commons import read_yaml, create_directories

In [69]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.feature_engineering

        create_directories([config.root_dir])

        data_preprocessing_config = DataPreprocessingConfig(
            root_dir=config.root_dir,
            training_data_path=config.training_data_path,
            training_data_file=config.training_data_file,
            training_cleansed_data=config.training_cleansed_data
        )

        return data_preprocessing_config

In [37]:
# from textClassifier import logger
# import zipfile
# import os
# from textClassifier.utils.commons import get_size
# import gdown
# import spacy
# from nltk.corpus import stopwords
# import string



# class DataPreprocessing:
#     def __init__(self, config: DataPreprocessingConfig):
#         self.config = config

#     def preprocess_text(self):
#         nlp = spacy.load('en_core_web_sm')
#         stop_words = set(stopwords.words('english'))
#         text = text.lower() # Convert to lowercase

#         # Remove punctuation
#         text = text.translate(str.maketrans('', '', string.punctuation))

#         # Tokenize and lemmatize
#         doc = nlp(text)
#         tokens = [token.lemma_ for token in doc if token.text not in stop_words]
#         return ' '.join(tokens)

#     def pass_data_to_text_process(self):
#         # self.config.training_data_file
#         data['cleaned_text'] = data['text'].apply(preprocess_text)
#         return data


In [38]:
# dp = DataPreprocessing(config_manager)

C:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\artifacts\data_ingestion\Tweets.csv

In [39]:
# dp.preprocess_text('hi dude Happy was how are you doing')

'hi dude happy'

In [72]:
from textClassifier import logger
import zipfile
import os
from textClassifier.utils.commons import get_size
import gdown
import spacy
from nltk.corpus import stopwords
import string
import pandas as pd


class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config
        self.nlp = spacy.load('en_core_web_sm')
        self.stop_words = set(stopwords.words('english'))


    def text_process(self, text: str) -> str:
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if token.text not in self.stop_words]
        return ' '.join(tokens)
    
    def get_data_from_features(self) -> pd.DataFrame:
        data = pd.read_csv(self.config.training_data_file)
        data['cleaned_text'] = data['text'].apply(self.text_process)
        return data
    
    def save_data(self, data: pd.DataFrame, path_name: str='cleaned_tweets.csv') -> None:
        out_path_name = Path(self.config.training_cleansed_data)
        data.to_csv(out_path_name, index=False)

        





        

In [80]:
config_manager = ConfigurationManager()
get_data_pre_config = config_manager.get_data_preprocessing_config()
data_processing = DataPreprocessing(config = get_data_pre_config)
output = data_processing.get_data_from_features()
save_data = data_processing.save_data()

[2025-02-08 15:30:36,292: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-02-08 15:30:36,297: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-02-08 15:30:36,299: INFO: commons: created directory at: artifacts]
[2025-02-08 15:30:36,301: INFO: commons: created directory at: artifacts/feature_engineering]


TypeError: save_data() missing 1 required positional argument: 'data'

In [79]:
print(output.head())

             tweet_id airline_sentiment  airline_sentiment_confidence  \
0  570306133677760513           neutral                        1.0000   
1  570301130888122368          positive                        0.3486   
2  570301083672813571           neutral                        0.6837   
3  570301031407624196          negative                        1.0000   
4  570300817074462722          negative                        1.0000   

  negativereason  negativereason_confidence         airline  \
0            NaN                        NaN  Virgin America   
1            NaN                     0.0000  Virgin America   
2            NaN                        NaN  Virgin America   
3     Bad Flight                     0.7033  Virgin America   
4     Can't Tell                     1.0000  Virgin America   

  airline_sentiment_gold        name negativereason_gold  retweet_count  \
0                    NaN     cairdin                 NaN              0   
1                    NaN    jnar

In [81]:
from textClassifier import logger
import pandas as pd
import spacy
from nltk.corpus import stopwords
import string
from pathlib import Path

class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config
        self.nlp = spacy.load('en_core_web_sm')  # Load spaCy model
        self.stop_words = set(stopwords.words('english'))  # Load stopwords

    def text_process(self, text: str) -> str:
        """
        Preprocesses the input text by:
        1. Converting to lowercase
        2. Removing punctuation
        3. Tokenizing and lemmatizing
        4. Removing stopwords
        """
        # Convert to lowercase
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize and lemmatize
        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if token.text not in self.stop_words]

        return ' '.join(tokens)

    def get_data_from_features(self) -> pd.DataFrame:
        """
        Loads the dataset from the path specified in the config and applies preprocessing to the text column.
        """
        # Check if the file exists
        if not Path(self.config.training_data_file).exists():
            raise FileNotFoundError(f"File not found: {self.config.training_data_file}")

        # Load the dataset
        data = pd.read_csv(self.config.training_data_file)

        # Apply preprocessing to the text column
        data['cleaned_text'] = data['text'].apply(self.text_process)

        return data

    def save_data(self, data: pd.DataFrame, path_name: str = 'cleaned_tweets.csv') -> None:
        """
        Saves the preprocessed data to the specified output file.

        Args:
            data (pd.DataFrame): The preprocessed DataFrame to save.
            path_name (str): The name of the output file. Defaults to "cleaned_tweets.csv".
        """
        # Create the full output file path
        out_path_name = Path(self.config.training_cleansed_data) / path_name

        # Save the DataFrame to CSV
        data.to_csv(out_path_name, index=False)
        logger.info(f"Preprocessed data saved to {out_path_name}")

In [82]:
config_manager = ConfigurationManager()

# Get the data preprocessing config
get_data_pre_config = config_manager.get_data_preprocessing_config()

# Initialize DataPreprocessing with the config
data_processing = DataPreprocessing(config=get_data_pre_config)

# Load and preprocess the data
output = data_processing.get_data_from_features()

# Save the preprocessed data
data_processing.save_data(data=output, path_name='cleaned_tweets.csv')

[2025-02-08 15:41:41,083: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-02-08 15:41:41,087: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-02-08 15:41:41,090: INFO: commons: created directory at: artifacts]
[2025-02-08 15:41:41,094: INFO: commons: created directory at: artifacts/feature_engineering]
[2025-02-08 15:43:36,407: INFO: 1592871266: Preprocessed data saved to artifacts\feature_engineering\cleaned_tweets.csv]
